In [ ]:
import os
from geffnet import create_model
from torchvision import transforms, datasets
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from google.colab import files, drive


In [2]:
!pip install geffnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:

!unzip "/content/drive/MyDrive/dataset_prepared.zip" -d dataset_prepared


Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f073.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f074.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f075.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f077.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f078.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f079.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f080.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f081.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f082.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f083.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fresh/p_f084.png  
  inflating: dataset_prepared/dataset_prepared/train/potato/fre

In [6]:
DATA_DIRECTORY = "/content/dataset_prepared/dataset_prepared"
train_directory = DATA_DIRECTORY + "/train"
test_directory = DATA_DIRECTORY + "/test"
val_directory = DATA_DIRECTORY + "/val"

In [7]:
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomRotation(30),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                [0.229, 0.224, 0.225])
        ])
    }


In [8]:
class FruitConditionDataset(Dataset):
    def __init__(self, root_dir, transform=None, label2id=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for fruit in os.listdir(root_dir):
            fruit_path = os.path.join(root_dir, fruit)
            if not os.path.isdir(fruit_path):
                continue
            for condition in os.listdir(fruit_path):
                condition_path = os.path.join(fruit_path, condition)
                if not os.path.isdir(condition_path):
                    continue
                label_name = f"{fruit}/{condition}"
                label_id = label2id[label_name]
                for img_name in os.listdir(condition_path):
                    if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
                        img_path = os.path.join(condition_path, img_name)
                        self.samples.append((img_path, label_id))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [9]:
!ls /content/dataset_prepared/dataset_prepared/


test  train  val


In [ ]:
data_root = test_directory

label2id = {}
counter = 0

for fruit in os.listdir(data_root):
    fruit_path = os.path.join(data_root, fruit)
    if not os.path.isdir(fruit_path):
        continue
    for condition in os.listdir(fruit_path):
        label = f"{fruit}/{condition}"
        label2id[label] = counter
        counter += 1

print(len(label2id))


10


In [ ]:
train_dataset = FruitConditionDataset(
    root_dir=train_directory,
    transform=data_transforms['train'],
    label2id=label2id
)

val_dataset = FruitConditionDataset(
    root_dir=val_directory,
    transform=data_transforms['val'],
    label2id=label2id
)

In [12]:
batch_size = 256

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=2)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 10
save_path = '/content/drive/MyDrive/fruit_model/efficientnet_fruits.pth' # where we store model

os.makedirs(os.path.dirname(save_path), exist_ok=True)

def get_model(num_classes, pretrained=True):
    """
    Creates EfficientNet B3 and changes classifier to num_classes
    """
    model = create_model('efficientnet_b3', pretrained=pretrained)

    for param in model.parameters():
        param.requires_grad = False

    in_features = model.classifier.in_features
    model.classifier = nn.Linear(in_features, num_classes)

    for param in model.classifier.parameters():
        param.requires_grad = True

    return model.to(device)

def train_model(model, train_loader, val_loader, num_epochs=5, lr=1e-4):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total

        # --- Validation ---
        model.eval()
        val_correct, val_total = 0, 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_acc = 100 * val_correct / val_total
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.2f}%")

        # Saves the best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("Model saved:", save_path)

        scheduler.step()

    print("Train loop is finished. Best Accuracy:", best_acc)
    return model

model = get_model(num_classes=num_classes, pretrained=True)
trained_model = train_model(model, train_loader, val_loader, num_epochs=10, lr=1e-4)


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


Epoch 1/10 - Training: 100%|██████████| 36/36 [00:39<00:00,  1.10s/it]


Epoch [1/10] | Train Loss: 2.2091 | Train Acc: 24.23% | Val Loss: 2.0843 | Val Acc: 44.84%
Model saved: /content/drive/MyDrive/fruit_model/efficientnet_fruits.pth


Epoch 2/10 - Training: 100%|██████████| 36/36 [00:38<00:00,  1.08s/it]


Epoch [2/10] | Train Loss: 2.0271 | Train Acc: 49.24% | Val Loss: 1.8964 | Val Acc: 64.79%
Model saved: /content/drive/MyDrive/fruit_model/efficientnet_fruits.pth


...

Epoch [8/10] | Train Loss: 1.4145 | Train Acc: 76.25% | Val Loss: 1.2907 | Val Acc: 80.89%
Model saved: /content/drive/MyDrive/fruit_model/efficientnet_fruits.pth


Epoch 9/10 - Training: 100%|██████████| 36/36 [00:37<00:00,  1.04s/it]


Epoch [9/10] | Train Loss: 1.3770 | Train Acc: 75.54% | Val Loss: 1.2474 | Val Acc: 81.30%
Model saved: /content/drive/MyDrive/fruit_model/efficientnet_fruits.pth


Epoch 10/10 - Training: 100%|██████████| 36/36 [00:39<00:00,  1.11s/it]


Epoch [10/10] | Train Loss: 1.3369 | Train Acc: 76.04% | Val Loss: 1.2003 | Val Acc: 81.82%
Model saved: /content/drive/MyDrive/fruit_model/efficientnet_fruits.pth
Train loop is finished. Best Accuracy: 81.82291666666667
